# Walkthrough 5: Experimentation and batch jobs #
 

## Introduction ##

Multiple jobs are needed to either average results or to scan control parameters to map out dependencies and trends.  In this walkthrough, we will explore ways of submitting and managing lists of *QuantumMatter* objects.  

### Imports and user authentication ###

In [ ]:
import matplotlib.pyplot as plt
from oqtant.schemas.quantum_matter import QuantumMatterFactory

qmf = QuantumMatterFactory()
qmf.get_login()

In [ ]:
qmf.get_client()

## Submit a list of QuantumMatter objects to generate many independent jobs ##

### Create a list of QuantumMatter objects ###

Make a list of *QuantumMatter* objects that each have a different target temperature:

In [ ]:
N = 2
matters = [
    qmf.create_quantum_matter(
        temperature=50 * (n + 1), name="quantum matter run " + str(n + 1) + "/" + str(N)
    )
    for n in range(N)
]

list(map(type, matters))

### Submit the list to Oqtant QMS###

Submit many independent jobs to be run on the hardware. Each of these jobs enters Oqtant's queueing system at nearly the same time, so they will *likely* be executed near each other in time, depending on current queue usage.

In [ ]:
[matter.submit(track=True) for matter in matters]

### Access job results ###

Once all our submitted jobs are complete, we access the results in the same way as if we had submitted the programs individually:

In [ ]:
# retrieve jobs from server one at a time, creating corresponding local job objects
[matter.get_result() for matter in matters]

# access the results individually and plot them together
lns = []
lbls = []
plt.figure(figsize=(5, 4))
for matter in matters:
    (ln,) = plt.plot(matter.output.get_slice(axis="x"))
    plt.xlabel("x position (pixels)")
    plt.ylabel("OD")
    lns.append(ln)
    lbls.append(str(matter.output.temperature_nk))
plt.legend(lns, lbls, loc="upper right", title="temp (nK)")
plt.show()

plt.figure(figsize=(5, 4))
plt.plot(
    [matter.output.temperature_nk for matter in matters],
    [
        100 * matter.output.condensed_atom_number / matter.output.tof_atom_number
        if matter.output.tof_atom_number > 0
        else 0
        for matter in matters
    ],
    "-o",
)
plt.xlabel("temperature (nK)")
plt.ylabel("condensed fraction (%)")
plt.show()

## Generate and submit a "batch" job ##

There is also the option to submit a list of *matter* objects as a single *batch* job, which guarantees that the sequence executes sequentially on the hardware.  This feature is useful for detailed experimentation or investigation, where subsequent shots need to be compared to each other in detail.  Using sequential hardware shots reduces system drift or inconsistency.  

In the case of bundling into a single batch job, only one job id will be generated.  Programmatically, the batch job will be composed of multiple *run*s on Oqtant hardware, and retrieving job results will require specifying the run number $1 \ldots N$ when fetching the job results, where there were $N$ runs in the job.    

*NOTE: The resulting name of a batch job will default to the name given to the first QuantumMatter object in the provided list.  Alternatively, a global name can be provided at the point of submission to the client.*  

*NOTE: Each individual *run* is charged against your job quota.  A single batch job will naturally contain multiple runs.*

### Create a list of QuantumMatter objects to submit as a batch ###

In [ ]:
# create a list of QuantumMatter objects
N = 2
matters = [
    qmf.create_quantum_matter(
        temperature=50 * (n + 1),
        lifetime=20 + (n * 2),
        time_of_flight=3 + (n * 2),
        note=str(n + 1)
        + "/"
        + str(N),  # notes persist in batches and will remain tied to each matter object
    )
    for n in range(N)
]

### Submit the list as a batch job ###

Submit our list of *matter* objects to generate a batch job using the `QuantumMatterFactory.submit_list_as_batch()` method:

In [ ]:
# submit the list as a batch that will run sequentially on the hardware
# returns only a single job id, e.g., "1cdb4ff7-c5ed-46d3-a667-8b12f0cd1349"
matter_batch = qmf.submit_list_as_batch(
    matter_list=matters, name="a batch!", track=True  # global batch name
)

### Access batch job results ###

Retrieve batch job run results one at a time by specifying the desired run number using `QuantumMatterFactory.get_batch_result()` method.  If omitted (as for non-batch jobs with just a single run), the data for the first run will be fetched.  

*NOTE: The added complication of managing multiple runs within a single job is is an unavoidable consequence of ensuring that the runs execute sequentially on the hardware.*

In [ ]:
# create local jobs based on single runs of the batch job
first_run = qmf.get_batch_result(matter_batch, run=1)
second_run = qmf.get_batch_result(matter_batch, run=2)

In [ ]:
print(f"Run: {first_run.run} of {len(matters)}")
print(f"Job Name: {first_run.name}")
print(f"Job Note: {first_run.note}")
print(f"Input Values: {first_run.input}")
print(f"Temperature: {first_run.output.temperature_nk}")

print(f"\nRun: {second_run.run} of {len(matters)}")
print(f"Job Name: {second_run.name}")
print(f"Job Note: {first_run.note}")
print(f"Input Values: {second_run.input}")
print(f"Temperature: {second_run.output.temperature_nk}")

Plot the results together as above, but augment our approach at extracting the data:

In [ ]:
my_runs = []
for n in range(N):
    my_runs.append(qmf.get_batch_result(matter_batch, run=n + 1))

lns = []
lbls = []
plt.figure(figsize=(5, 4))
for run in my_runs:
    (ln,) = plt.plot(run.output.get_slice(axis="x"))
    plt.xlabel("x position (pixels)")
    plt.ylabel("OD")
    lns.append(ln)
    lbls.append(str(run.output.temperature_nk))
plt.legend(lns, lbls, loc="upper right", title="temp (nK)")
plt.show()

plt.figure(figsize=(5, 4))
plt.plot(
    [run.output.temperature_nk for run in my_runs],
    [
        100 * run.output.condensed_atom_number / run.output.tof_atom_number
        if run.output.tof_atom_number > 0
        else 0
        for run in my_runs
    ],
    "-o",
)
plt.xlabel("temperature (nK)")
plt.ylabel("condensed fraction (%)")
plt.show()

### Saving and loading batch job results ###

When we fetch results from a batch job, our instantiated local job only contains output data for the specified run.  However, fetching multiple runs will result in many local jobs that share the same job id.  

When saving the data associated with these jobs, the job id (which is used as the default filename) is not unique.  

In [ ]:
first_run.write_to_file()
second_run.write_to_file()

In this case, the `QuantumMatterFactory.write_job_to_file()` method automatically appends the run information in the format *id_run_n_of_N.txt*.